In [1]:
import random

### constants ###
corner='+'
vwall='|'
hwall='---'
empty='   '
begin_symbol=' S '
end_symbol = ' E '
dirs={(1,0):'down',(-1,0):'up',(0,-1):'left',(0,1):'right'}

## dicts for convertion
vwalldict={True:vwall,False:" "}
hwalldict={True:hwall,False:empty}
opositedict={'up':'down','down':'up','left':'right','right':'left'}

class cell:
    def __init__(self):
        self.upper=True
        self.lower=True
        self.left=True
        self.right=True
        self.center=empty
        self.visited=False
        
    def remove_wall(self,direction):
        if direction =='left': self.left=False
        elif direction =='right': self.right=False
        elif direction =='up': self.upper=False
        elif direction =='down': self.lower=False
        else: print 'incorrect wall direction', direction
        
class maze:
    def __init__(self, rows,cols):
        self.rows=rows
        self.cols=cols
        self.cells={}
        self.current = (-1,-1) # will be set to begin 
        self.unvisited = set([]) # will be initialised when making start and end

        for row in range(rows):
            for col in range(cols):
                self.cells[(row,col)]=cell()
                
        self.make_start_and_end()
                
    def print_cells_upper_parts(self,row):
        print corner+corner.join([hwalldict[self.cells[(row,cellcol)].upper] for cellcol in range(self.cols)])+corner

    def print_cells_middle_parts(self,row):
        walls=[vwalldict[self.cells[(row,cellcol)].left] for cellcol in range(self.cols)]+[vwalldict[self.cells[(row,self.cols-1)].right]]
        centers=[self.cells[(row,cellcol)].center for cellcol in range(self.cols)]
        result = [None]*(2*self.cols+1)
        result[::2]=walls
        result[1::2]=centers
        print "".join(result)
        
    def print_maze(self):
        for row in range(self.rows):
               self.print_cells_upper_parts(row)
               self.print_cells_middle_parts(row)
        self.print_cells_upper_parts(0)
        
    def make_start_and_end(self):
        all_coords=[(row,col) for row in range(self.rows) for col in range(self.cols)]
        self.unvisited=set(all_coords)
        border=filter(lambda x: x[0]==0 or x[0] == (self.rows-1) or x[1]==0 or x[1] == (self.cols-1),all_coords)
        begin = random.choice(border)
        border.remove(begin)
        end=random.choice(border)
        self.cells[begin].center=begin_symbol
        self.mark_visited(begin)
        self.cells[end].center=end_symbol
        self.current=begin
        
    def neighbors(self,coords):        
        coords = [((dir[0]+coords[0],dir[1]+coords[1]),dirs[dir]) for dir in dirs]
        filtered = filter(lambda x: (x[0][0] >=0) and (x[0][0] <self.rows) and (x[0][1] <self.cols) and (x[0][1] >=0),coords)
        return filtered
    
    def unvisited_neighbors(self,coords):
        return filter(lambda x: not self.cells[x[0]].visited, self.neighbors(coords))

    def mark_visited(self, coords):
        if not self.cells[coords].visited:
            self.cells[coords].visited=True
            self.unvisited.remove(coords)
    
    def grow_path(self):
        options=self.unvisited_neighbors(self.current)
        while options:
            option=random.choice(options)
            self.cells[self.current].remove_wall(option[1])
            self.cells[option[0]].remove_wall(opositedict[option[1]])
            self.mark_visited(self.current)
            self.current=option[0]
            if self.cells[self.current].center==end_symbol:
                self.mark_visited(self.current)
                break
            options=self.unvisited_neighbors(self.current)            
        self.mark_visited(self.current)
        
    def find_start_new_path(self):
        for option in list(self.unvisited):
            for neighbor in [x[0] for x in self.neighbors(option)]:
                if self.cells[neighbor].visited:
                    symbol=self.cells[neighbor].center
                    if symbol == empty:
                        self.current=neighbor
                        return
                
    def make_all_paths(self):
        while self.unvisited:          
            self.grow_path()
            self.find_start_new_path() 
            
my_maze=maze(10,15)
my_maze.make_all_paths()
my_maze.print_maze()

+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|   |                       |       |           |           |
+   +   +---+   +---+---+---+   +   +---+   +   +   +   +---+
|       |   |       |           |       |   |       |       |
+---+   +   +---+   +---+   +---+---+   +---+---+---+---+   +
|   |       |   |       |   |       |       |               |
+   +---+   +   +---+   +   +   +   +---+   +---+   +---+---+
|       |   |       |   |   |   |       |       |   |       |
+---+   +   +---+   +   +   +   +---+   +---+   +   +---+   +
|           |       |   |   |       |   |           |       |
+---+---+---+---+   +   +   +---+   +   +---+---+---+   +   +
| S |           |       |           |           |       |   |
+   +   +---+   +---+   +---+---+---+---+---+   +   +---+   +
|   |       |           |                       |       |   |
+   +---+   +---+---+---+   +---+   +---+---+   +---+---+   +
|       |                   |   |   |                   |   |
+---+   